In [1]:
pip install transformers datasets torch pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import pandas as pd

# Load your Excel dataset
df = pd.read_excel("/content/guvi_qa_table.xlsx")

# Inspect columns
print(df.head())


                                        Question  \
0                                  What is GUVI?   
1                 What is the main goal of GUVI?   
2            What domains do GUVI courses cover?   
3  What are the types of learning modes in GUVI?   
4                  What are GUVI's free courses?   

                                              Answer  
0  GUVI is an Indian ed-tech platform started in ...  
1  To democratize tech education for non-English-...  
2  Programming, Web Development, AI/ML, Cybersecu...  
3  Self-paced courses, Zen Class live programs, a...  
4  Python, Java, ChatGPT & AI, JavaScript for Beg...  


In [3]:
df["text"] = "Q: " + df["Question"].astype(str) + "\nA: " + df["Answer"].astype(str)

# Preview
print(df["text"].head())


0    Q: What is GUVI?\nA: GUVI is an Indian ed-tech...
1    Q: What is the main goal of GUVI?\nA: To democ...
2    Q: What domains do GUVI courses cover?\nA: Pro...
3    Q: What are the types of learning modes in GUV...
4    Q: What are GUVI's free courses?\nA: Python, J...
Name: text, dtype: object


In [4]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["text"]])
print(dataset)


Dataset({
    features: ['text'],
    num_rows: 561
})


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Add pad token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()   # ✅ important for loss calculation
    return tokens

tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/561 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [7]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [8]:
# Split dataset
dataset_split = dataset.train_test_split(test_size=0.1)

# Tokenize with labels
def tokenize(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_split = dataset_split.map(tokenize, batched=True, remove_columns=["text"])


Map:   0%|          | 0/504 [00:00<?, ? examples/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

In [9]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


CUDA available: True
Device: cuda


In [10]:
import torch
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./guvi-gpt2",
    overwrite_output_dir=True,
    do_eval=True,
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=1,      # one pass through dataset
    max_steps=200,           # stop early after 200 steps
    weight_decay=0.01,
    save_total_limit=1,
    save_strategy="steps",
    save_steps=100,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=20,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_split["train"],
    eval_dataset=tokenized_split["test"],
    tokenizer=tokenizer,
)

/tmp/ipython-input-4172515846.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
20,4.945000
40,0.410400
60,0.313500
80,0.293700
100,0.297800
120,0.264800
140,0.288500
160,0.275800
180,0.292200
200,0.298900


TrainOutput(global_step=200, training_loss=0.7680581140518189, metrics={'train_runtime': 113.6803, 'train_samples_per_second': 3.519, 'train_steps_per_second': 1.759, 'total_flos': 52258406400000.0, 'train_loss': 0.7680581140518189, 'epoch': 0.7936507936507936})

In [12]:
trainer.save_model("./guvi-gpt2-model")
tokenizer.save_pretrained("./guvi-gpt2-model")


('./guvi-gpt2-model/tokenizer_config.json',
 './guvi-gpt2-model/special_tokens_map.json',
 './guvi-gpt2-model/vocab.json',
 './guvi-gpt2-model/merges.txt',
 './guvi-gpt2-model/added_tokens.json',
 './guvi-gpt2-model/tokenizer.json')

In [13]:
import shutil

shutil.make_archive("guvi-gpt2-model", 'zip', "./guvi-gpt2-model")


'/content/guvi-gpt2-model.zip'

In [ ]:
from google.colab import files
files.download("guvi-gpt2-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
!ls /content/drive/MyDrive


'Colab Notebooks'  'guvi-gpt2-model (4).zip'  'panel test.zip'
 dataset	    moviessmall.zip	       solar
 dataset.zip	   'panel test'


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "./guvi-gpt2-model"  # or extracted zip folder
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)


OSError: Can't load tokenizer for './guvi-gpt2-model'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './guvi-gpt2-model' is the correct path to a directory containing all relevant files for a GPT2Tokenizer tokenizer.

In [16]:
!pip install transformers torch deep-translator langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 56.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=7f623e6ed9b3aabcfd7fe50ec10bcccb45fdc9d289dfe5fdb72bc805aad521f7
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [17]:
from deep_translator import GoogleTranslator
from langdetect import detect
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# =========================
# 1. Translation Functions
# =========================
def to_english(text):
    lang = detect(text)
    if lang == "en":
        return text, "en"
    translated = GoogleTranslator(source=lang, target="en").translate(text)
    return translated, lang

def from_english(text, target_lang):
    if target_lang == "en":
        return text
    return GoogleTranslator(source="en", target=target_lang).translate(text)

# =========================
# 2. Load GPT-2 Model
# =========================
model_path = "/content/guvi-gpt2-model"   # change if needed
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# =========================
# 3. Text Generation
# =========================
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# =========================
# 4. Multilingual Chat
# =========================
def multilingual_chat(user_input):
    # Translate user input → English
    eng_input, lang = to_english(user_input)

    # Generate GPT-2 response (English)
    eng_response = generate_response(eng_input)

    # Translate back → Original Language
    final_response = from_english(eng_response, lang)

    return final_response

# =========================
# 5. Test
# =========================
print(multilingual_chat("வணக்கம், நீங்கள் என்ன செய்கிறீர்கள்?"))  # Tamil
print(multilingual_chat("Hola, ¿qué tal?"))                     # Spanish
print(multilingual_chat("Hello, how are you?"))                 # English


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


வணக்கம், நீங்கள் என்ன செய்கிறீர்கள்?
ப: நான் ஜாவாவில் எனது போக்கை மேம்படுத்த முயற்சிக்கிறேன்.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hola, ¿Qué Tal? ¿
Hello, how are you?
A: For courses with multiple enrollments, you must complete multiple assignments in a row, or you will not earn points.


In [18]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.2 MB/s eta 0:00:00


In [19]:
!pip install pyngrok

In [20]:
!pip install streamlit-lottie



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 47.7 MB/s eta 0:00:00


In [21]:
%%writefile app.py
import streamlit as st
from deep_translator import GoogleTranslator
from langdetect import detect
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch, base64, requests
from streamlit_lottie import st_lottie

# =========================
# Translation Functions
# =========================
def to_english(text):
    lang = detect(text)
    if lang == "en":
        return text, "en"
    translated = GoogleTranslator(source=lang, target="en").translate(text)
    return translated, lang

def from_english(text, target_lang):
    if target_lang == "en":
        return text
    return GoogleTranslator(source="en", target=target_lang).translate(text)

# =========================
# Load GPT-2 Model
# =========================
@st.cache_resource
def load_model():
    model_path = "./guvi-gpt2-model"
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    return tokenizer, model, device

tokenizer, model, device = load_model()

# =========================
# Text Generation
# =========================
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# =========================
# Multilingual Chat Function
# =========================
def multilingual_chat(user_input):
    eng_input, lang = to_english(user_input)

    # Check if it's a course query
    course = recommend_course(eng_input)
    if course:
        eng_response = f"I recommend you check out GUVI's course: {course}"
    else:
        eng_response = generate_response(eng_input)

    final_response = from_english(eng_response, lang)
    return final_response

# =========================
# Simple Course Recommendation Logic
# =========================
def recommend_course(text):
    text = text.lower()
    if "python" in text:
        return "🐍 Python Programming – Beginner to Advanced"
    elif "data science" in text:
        return "📊 Data Science Master Program"
    elif "ai" in text or "machine learning" in text:
        return "🤖 AI & Machine Learning with Real Projects"
    elif "web" in text or "full stack" in text:
        return "🌐 Full Stack Web Development"
    elif "java" in text:
        return "☕ Java Programming Essentials"
    else:
        return None

# =========================
# Helpers
# =========================
def image_to_base64(image_path):
    with open(image_path, "rb") as f:
        data = f.read()
    return base64.b64encode(data).decode()

def load_lottieurl(url: str):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

# =========================
# Streamlit UI Config
# =========================
st.set_page_config(page_title="🌐 GUVI Multilingual GPT-2 Chatbot", layout="wide")

# Background + Styles
st.markdown(
    """
    <style>
    .stApp { background-color: #f0fff4; }
    .user-bubble {
        background-color: #2ecc71;
        color: white;
        padding: 10px;
        border-radius: 15px;
        margin: 5px;
        text-align: right;
        animation: fadeInRight 1s;
    }
    .bot-bubble {
        background-color: #ecf0f1;
        color: black;
        padding: 10px;
        border-radius: 15px;
        margin: 5px;
        text-align: left;
        animation: fadeInLeft 1s;
    }
    @keyframes fadeInRight {
        from {opacity: 0; transform: translateX(50px);}
        to {opacity: 1; transform: translateX(0);}
    }
    @keyframes fadeInLeft {
        from {opacity: 0; transform: translateX(-50px);}
        to {opacity: 1; transform: translateX(0);}
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Load Animations
lottie_ai = load_lottieurl("https://assets10.lottiefiles.com/packages/lf20_ysrn2f.json")
lottie_chat = load_lottieurl("https://assets2.lottiefiles.com/packages/lf20_m9ubnc.json")

# =========================
# 🚀 Redirect if "Get Started" clicked
# =========================
if "go_chat" in st.session_state and st.session_state["go_chat"]:
    st.session_state["go_chat"] = False
    st.markdown("<h2 style='text-align:center;'>💬 Start Chatting</h2>", unsafe_allow_html=True)

    if "messages" not in st.session_state:
        st.session_state["messages"] = []

    user_input = st.text_input("💬 Type your message:", placeholder="Say something like 'I want to learn Python'...")

    if user_input:
        st.session_state["messages"].append(("user", user_input))
        with st.spinner("🤖 Thinking..."):
            bot_reply = multilingual_chat(user_input)
        st.session_state["messages"].append(("bot", bot_reply))

    if st.button("🗑️ Clear Chat"):
        st.session_state["messages"] = []
        st.rerun()   # ✅ FIXED

    if lottie_chat:
        st_lottie(lottie_chat, height=100, key="chat-anim")

    for sender, msg in st.session_state["messages"]:
        if sender == "user":
            st.markdown(f"<div class='user-bubble'>{msg}</div>", unsafe_allow_html=True)
        else:
            st.markdown(f"<div class='bot-bubble'>🤖 {msg}</div>", unsafe_allow_html=True)

    st.stop()  # ✅ Prevent rendering other tabs

# =========================
# Tabs
# =========================
tab1, tab2, tab3 = st.tabs(["🏠 Home", "💬 Chatbot", "ℹ️ About"])

# -------------------------
# Home Tab
# -------------------------
with tab1:
    col1, col2, col3 = st.columns([1,3,1])
    with col2:
        st.image("guvi logo.jpg", width=150)
    st.markdown("<h1 style='text-align:center; color:#27ae60;'>🌐 GUVI Multilingual GPT-2 Chatbot</h1>", unsafe_allow_html=True)

    # Arun Sir Image (only in Home)
    img_b64 = image_to_base64("arun sir.jpg")
    st.markdown(
        f"""
        <div style="position: fixed; top: 200px; right: 20px; z-index: 999;">
            <img src="data:image/png;base64,{img_b64}" width="120">
        </div>
        """,
        unsafe_allow_html=True
    )

    if lottie_ai:
        st_lottie(lottie_ai, height=250, key="ai-home")

    st.markdown("<h4 style='text-align:center; color:#555;'>🤖 Breaking language barriers with AI-powered conversations</h4>", unsafe_allow_html=True)

    # Extra Welcome Message
    st.markdown(
        """
        <div style="text-align:center; font-size:18px; color:#34495e; margin-top:20px;">
        ✨ Welcome to the <b>GUVI Multilingual Chatbot</b>! <br>
        🌍 Ask me anything in your own language.<br>
        🎯 I’ll translate, think, and reply back instantly.<br>
        🚀 Start learning, exploring, and discovering new opportunities today!
        </div>
        """,
        unsafe_allow_html=True
    )

    # ✅ Working Get Started Button
    if st.button("🚀 Get Started with Chatbot"):
        st.session_state["go_chat"] = True
        st.rerun()   # ✅ FIXED

# -------------------------
# Chatbot Tab
# -------------------------
with tab2:
    st.markdown("<h2 style='text-align:center;'>💬 Start Chatting</h2>", unsafe_allow_html=True)

    if "messages" not in st.session_state:
        st.session_state["messages"] = []

    user_input = st.text_input("💬 Type your message:", placeholder="Say something like 'I want to learn Python'...")

    if user_input:
        st.session_state["messages"].append(("user", user_input))
        with st.spinner("🤖 Thinking..."):
            bot_reply = multilingual_chat(user_input)
        st.session_state["messages"].append(("bot", bot_reply))

    if st.button("🗑️ Clear Chat"):
        st.session_state["messages"] = []
        st.rerun()   # ✅ FIXED

    if lottie_chat:
        st_lottie(lottie_chat, height=100, key="chat-anim")

    for sender, msg in st.session_state["messages"]:
        if sender == "user":
            st.markdown(f"<div class='user-bubble'>{msg}</div>", unsafe_allow_html=True)
        else:
            st.markdown(f"<div class='bot-bubble'>🤖 {msg}</div>", unsafe_allow_html=True)

# -------------------------
# About Tab
# -------------------------
with tab3:
    st.markdown("<h2>ℹ️ About the Project</h2>", unsafe_allow_html=True)
    st.write("""
    ### Project: GUVI Multilingual GPT-2 Chatbot
    - 🌍 Supports **multiple languages** using translation
    - 🤖 Powered by **fine-tuned GPT-2** for domain-specific responses
    - 🎨 Built with **Streamlit** for an interactive dashboard
    - 🚀 Deployment-ready for Hugging Face / Streamlit Cloud

    ### Business Use Cases:
    - 🟢 **Customer Support Automation**
    - 🟢 **E-Learning Assistance**
    - 🟢 **Career Guidance**
    - 🟢 **Course Recommendations**

    ### Credits:
    - Built by: **k.Rajeshwari**
    - Guided by: **Arun Sir**
    - Platform: **GUVI**
    """)


Writing app.py


In [22]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 3s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.91.252.237
⠙⠹your url is: https://blue-apples-swim.loca.lt


In [23]:
from huggingface_hub import create_repo, upload_folder

# 🔑 Your Hugging Face token (with Write access)
hf_token = ""

# ⚡ Repo under your username
repo_id = "deekshu16/Guvi-multilingual-chatbot"

# 1. Create repo (safe even if it already exists)
create_repo(repo_id, private=False, exist_ok=True, token=hf_token)

# 2. Upload model folder (make sure it exists in Colab)
upload_folder(
    repo_id=repo_id,
    folder_path="./guvi-gpt2-model",   # trainer.save_model() saved here
    commit_message="Upload GUVI fine-tuned GPT-2 model",
    token=hf_token
)

print(f"✅ Model uploaded! Check: https://huggingface.co/{repo_id}")


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...t/guvi-gpt2-model/training_args.bin: 100%|##########| 5.30kB / 5.30kB            

  ...t/guvi-gpt2-model/model.safetensors:   0%|          | 1.10MB /  498MB            

✅ Model uploaded! Check: https://huggingface.co/deekshu16/Guvi-multilingual-chatbot


In [ ]:
#https://huggingface.co/spaces/deekshu16/Guvi-multilingual-chatbot